In [80]:
import sys
import os
import string

DATA_PATH = r"E:\books-2nd quarter\IR\Selma\Selma"
WORD_FREQUENCY_CSV_FILENAME = "Frequencies.csv"
INVERSE_DOCUMENT_FREQUENCY_CSV_FILENAME = "InverseDocumentFrequencies.csv"
TF_IDF_CSV_FILENAME = "tfidf.csv"

In [96]:

textdirs = os.listdir(DATA_PATH)

delimiter_chars=",.;:!?"
num_lines = 0
num_words = 0
num_chars = 0

counts = defaultdict(int)
docs = {}

word_occurrences = {}
word_count = {}
#print(type(word_occurrences))
for file in textdirs:
    #print("filename: " + file)
    #print(type(file))
    filename = os.path.join(DATA_PATH, file)
    txt_fil = open(filename, "r")

    pos = 0

    for lin in txt_fil:
        lin = lin.lower()
        words = lin.split()

        words2 = [ word.strip(delimiter_chars) for word in words ]

        for word in words2:
                counts[word] += 1
                pos += lin.find(word)
                if word in word_occurrences.keys():
                    #print(type(word))
                    if file in word_occurrences[word].keys():
                        word_occurrences[word][file].append(pos)
                    else:
                        word_occurrences[word][file] = [pos]
                else:
                    #word_occurrences[word] = { file.append([pos]) }
                    word_occurrences[word] = { file : [pos] }
        num_lines += 1
        num_words += len(words2)
        num_chars += len(lin)
        pos += len(lin) - 1
        word_count[file]=num_words
        
    relativefreqs = {}
    for word, rawCount in counts.items():
        if word.isalpha():
            relativefreqs[word] = rawCount / num_words
        counts[word] = 0
    docs[os.path.basename(file)] = relativefreqs

word_keys = word_occurrences.keys()

    #word_keys.sort()
word_keys = sorted(word_keys)
#print(word_occurrences)
#idx_fil = open("masterindex.idx", "w")
with open('masterindex.idx', 'w') as idx_fil:
    for word in word_keys:
        
        files = word_occurrences[word].keys()
        idx_fil.write(str(word) + ":")

        for f in files:
            idx_fil.write(str(f) + ":" )
            #values = word_occurrences[word][f]
           # print(type(values))
            #for v in values:
            idx_fil.write(str(word_occurrences[word][f]) + " ")
        idx_fil.write("\n")
        
txt_fil.close()
idx_fil.close()

df = pd.DataFrame(docs)
df = df.fillna(0)
df.to_csv("WORD_FREQUENCY_CSV.csv", encoding="utf-8") # write out to CSV
print("Done.")


Done.


In [92]:
import csv
import math
import pandas as pd

idf = {}

with open("WORD_FREQUENCY_CSV.csv", encoding="utf-8", newline='') as csvfile:
    wordsreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    
    #skip header information
    total_documents = len(next(wordsreader, None)) - 1
    for row in wordsreader:
        docsContainingWord = 0
        iterrow = iter(row)
        next(iterrow)
        for entry in iterrow:
            if (entry != '0.0') and (entry != '0'):
                docsContainingWord += 1
        idf[row[0]] = math.log(total_documents / (1 + docsContainingWord))
        
df = pd.DataFrame(idf, index=['Inverse Document Frequency'])
df2 = df.transpose()
df2.to_csv("INVERSE_DOCUMENT_FREQUENCY_CSV.csv", encoding="utf-8")
print("Done.")

Done.


In [93]:
import csv

file1reader = csv.reader(open("WORD_FREQUENCY_CSV.csv", encoding="utf-8"), delimiter=",")
file2reader = csv.reader(open("INVERSE_DOCUMENT_FREQUENCY_CSV.csv", encoding="utf-8"), delimiter=",")

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

#skip headers
header1 = next(file1reader) #header
header2 = next(file2reader) #header
with open("TF_IDF_CSV.csv", 'w', encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(header1)
    for row1, row2 in zip(file1reader, file2reader):
        rowOut = []
        iterrow1 = iter(row1)
        rowOut.append(next(iterrow1))
        for entry in iterrow1:
            entry = float(entry) * float(row2[1])
            rowOut.append(entry)
        writer.writerow(rowOut)
    
print("Done.")


Done.


In [94]:
df = pd.read_csv(TF_IDF_CSV_FILENAME, encoding="utf-8", index_col=0) 
df.rename(columns={'Unnamed: 0': 'Text'}, inplace=True) 
df = df.fillna(0) 
df.head(20)
df2 = df.transpose()
df2.head(20)

,a,abborrar,abborrarna,abborre,abborren,abborrens,abborrn,abcbok,abcdarierna,abessinien,...,övervåningen,övervåningens,överöste,övet,övning,övningar,övningarna,övre,övriga,övrigt
bannlyst.txt,2.483106e-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.324555e-05,0.000000,0.00002,0.000000,0.000000,0.000000,0.000000,0.000000e+00,7.759707e-06,3.103883e-06
gosta.txt,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.780542e-05,1.724092e-06,1.149394e-06
herrgard.txt,1.972230e-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.260199e-06,0.000006,0.00000,0.000000,0.000000,0.000005,0.000000,0.000000e+00,0.000000e+00,9.861148e-07
jerusalem.txt,1.475471e-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.888934e-06,0.000000,0.00000,0.000000,0.000003,0.000003,0.000004,0.000000e+00,1.475471e-06,0.000000e+00
kejsaren.txt,7.643683e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000003,0.000000,0.000003,0.000000,...,1.630938e-06,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,5.095788e-07
marbacka.txt,2.224941e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.297791e-06,2.224941e-07,4.449882e-07
nils.txt,0.000000e+00,0.000000,0.000000,0.000002,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.00000,0.000000,0.000002,0.000000,0.000000,2.787575e-06,1.781470e-06,4.858555e-07
osynliga.txt,9.241975e-07,0.000000,0.000000,0.000000,0.000000,0.000003,0.000000,0.000003,0.000000,0.000003,...,5.634189e-07,0.000000,0.00000,0.000002,0.000000,0.000000,0.000000,9.090076e-07,1.056226e-06,2.640564e-07
troll.txt,4.897866e-07,0.000003,0.000002,0.000000,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.225305e-07,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.264559e-06,2.448933e-07,0.000000e+00
